In [ ]:
#default_exp utils

In [ ]:
#exporti
import torch

In [ ]:
#exporti
def _infect_step(start_points):
    m, n, k = start_points.shape

    padded = torch.zeros((2 + m, 2 + n, 2 + k), dtype=start_points.dtype, device=start_points.device)
    padded[1:-1, 1:-1, 1:-1] = start_points

    max_of_neighbors = padded[1:-1, 1:-1, 1:-1] | \
                       padded[2:  , 1:-1, 1:-1] | \
                       padded[ :-2, 1:-1, 1:-1] | \
                       padded[1:-1, 2:  , 1:-1] | \
                       padded[1:-1,  :-2, 1:-1] | \
                       padded[1:-1, 1:-1, 2:  ] | \
                       padded[1:-1, 1:-1,  :-2]
    return max_of_neighbors

In [ ]:
#export
def infect(start_points, infectable):
    """
    An infection algorithm that starts from a set points and iteratively infects neighboring points.

    Returns
    -------
    torch.Tensor
    """
    infected_old = torch.zeros_like(start_points)
    assert start_points.dtype == infectable.dtype, f"{start_points.dtype} != {infectable.dtype}"
    assert start_points.device == infectable.device, f"{start_points.device} != {infectable.device}"
    infected = start_points.clone() & infectable

    while torch.any(infected_old != infected):
        infected_old = infected
        neighbors = _infect_step(infected)
        infected = neighbors & infectable
    return infected

# Tests

In [ ]:
import numpy as np

In [ ]:
%%time

def test_that_infect_does_not_disinfect():
    n, m, k = 10, 8, 2

    start_points = torch.tensor(np.random.rand(n, m, k).round(), dtype=bool)
    infectable = torch.tensor(np.random.rand(n, m, k).round(), dtype=bool) | start_points

    infected = infect(start_points, infectable)
    assert infected[start_points].all()


for _ in range(10):
    test_that_infect_does_not_disinfect()

CPU times: user 9.54 ms, sys: 210 µs, total: 9.75 ms
Wall time: 29.6 ms


In [ ]:
%%time

def test_that_only_infectable_parts_are_infected():
    n, m, k = 10, 4, 7

    start_points = torch.tensor(np.random.rand(n, m, k).round(), dtype=bool)
    infectable = torch.tensor(np.random.rand(n, m, k).round(), dtype=bool)

    infected = infect(start_points, infectable)
    assert not infected[~infectable].any()


for _ in range(10):
    test_that_only_infectable_parts_are_infected()

CPU times: user 11.1 ms, sys: 0 ns, total: 11.1 ms
Wall time: 11.1 ms


In [ ]:
%%time

def test_example0():
    n, m, k = 7, 3, 10

    start_points = torch.zeros(n, m, k, dtype=bool)
    start_points[0, 0, 0] = True

    infectable = torch.zeros(n, m, k, dtype=bool)
    infectable[0, 0, :] = True

    infected = infect(start_points, infectable)
    assert infected[0, 0, :].all()
    assert not infected[1:, 1:, :].any()


test_example0()

CPU times: user 1.95 ms, sys: 0 ns, total: 1.95 ms
Wall time: 4.11 ms


In [ ]:
%%time

def test_example1():
    n, m, k = 7, 3, 10

    start_points = torch.zeros(n, m, k, dtype=bool)
    start_points[0, 0, 0] = True
    start_points[2, 2, 0] = True

    infectable = torch.zeros(n, m, k, dtype=bool)
    infectable[0, 0, :] = True

    infected = infect(start_points, infectable)
    assert infected[0, 0, :].all()
    assert not infected[1:, 1:, :].any()


test_example1()

CPU times: user 3.01 ms, sys: 0 ns, total: 3.01 ms
Wall time: 2.56 ms


In [ ]:
%%time

def test_example2():
    n, m, k = 7, 3, 10

    start_points = torch.zeros(n, m, k, dtype=bool)
    start_points[0, 0, 0] = True
    start_points[2, 2, 0] = True

    infectable = torch.zeros(n, m, k, dtype=bool)
    infectable[0, 0, :] = True
    infectable[2, 2, 0] = True

    infected = infect(start_points, infectable)
    assert infected[0, 0, :].all()
    assert infected[2, 2, 0]
    assert infected[1:, 1:, :].any()


test_example2()

CPU times: user 2.22 ms, sys: 0 ns, total: 2.22 ms
Wall time: 4.33 ms


In [ ]:
%%time

def test_example3():
    n, m, k = 8, 8, 8

    start_points = torch.zeros(n, m, k, dtype=bool)
    start_points[0, 1, 0] = True

    infectable = torch.zeros(n, m, k, dtype=bool)
    infectable[0, 0, :] = True

    infected = infect(start_points, infectable)
    assert not infected.any()


test_example3()

CPU times: user 620 µs, sys: 0 ns, total: 620 µs
Wall time: 516 µs
